In [9]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

## Numerical perturbation analysis example

In [10]:
x = cp.Variable(2)
b= np.array([-2, -3, 1])
A = np.array([[1, 2],[1, -4],[5, 76]])
Q = np.array([[1, -0.5],[-0.5, 2]])
f = np.array([-1, 0])

In [11]:
constraints = [A@x <= b]
obj = cp.Minimize(cp.quad_form(x, Q)+f.T@x)
prob = cp.Problem(obj, constraints)
optimal_value = prob.solve()
print("status", prob.status)
print("optimal value", optimal_value)
print("x value", x.value)

status optimal
optimal value 8.222222222222221
x value [-2.33333333  0.16666667]


In [12]:
print("The dual variable values are:", constraints[0].dual_value)
lambdas = constraints[0].dual_value
print("KKT3.1", A[0]@x.value-b[0])
print("KKT3.2", A[1]@x.value-b[1])
print("KKT3.3", A[2]@x.value-b[2])
print("KKT4.1", 2*Q@x.value+f+A.T@lambdas)

The dual variable values are: [2.74774125 2.88523345 0.04007173]
KKT3.1 4.440892098500626e-16
KKT3.2 0.0
KKT3.3 5.329070518200751e-15
KKT4.1 [1.77635684e-15 0.00000000e+00]


In [13]:
perturbed_set = [-0.1, 0, 0.1]
for delta_1 in perturbed_set:
    for delta_2 in perturbed_set:
        print("delta1:", delta_1, "delta2:", delta_2)
        deltas = np.array([delta_1, delta_2])
        p_pred= optimal_value - lambdas[0:-1].T@deltas
        print("--The predicted optimal value is", p_pred)
        b = np.array([-2+delta_1, -3+delta_2, 1])
        constraints = [A@x <= b]
        obj = cp.Minimize(cp.quad_form(x, Q)+f.T@x)
        prob = cp.Problem(obj, constraints)
        p_exact = prob.solve()
        print("--The exact optimal value is", p_exact)

delta1: -0.1 delta2: -0.1
--The predicted optimal value is 8.785519691736578
--The exact optimal value is 8.815555555555559
delta1: -0.1 delta2: 0
--The predicted optimal value is 8.496996346890384
--The exact optimal value is 8.565
delta1: -0.1 delta2: 0.1
--The predicted optimal value is 8.208473002044192
--The exact optimal value is 8.318888888888889
delta1: 0 delta2: -0.1
--The predicted optimal value is 8.510745567068415
--The exact optimal value is 8.70638671875
delta1: 0 delta2: 0
--The predicted optimal value is 8.222222222222221
--The exact optimal value is 8.222222222222221
delta1: 0 delta2: 0.1
--The predicted optimal value is 7.933698877376028
--The exact optimal value is 7.980000000000002
delta1: 0.1 delta2: -0.1
--The predicted optimal value is 8.23597144240025
--The exact optimal value is 8.70638671875
delta1: 0.1 delta2: 0
--The predicted optimal value is 7.947448097554058
--The exact optimal value is 8.222222222222221
delta1: 0.1 delta2: 0.1
--The predicted optimal val

## Option price bounds

In [14]:
c = 1.15
f = 0.9
m = 200
S0 = 1
r = 1.05
y = cp.Variable(m)
S = np.linspace(0.5, 2, m)
p_collar = cp.Variable(1)
p = np.array([1, S0, 0.06, 0.03, 0.02, 0.01, p_collar])

In [15]:
value_call_1 = np.array([max(0, s-1.1) for s in S])
value_call_2 = np.array([max(0, s-1.2) for s in S])
value_put_1 = np.array([max(0, 0.8-s) for s in S])
value_put_2 = np.array([max(0, 0.7-s) for s in S])
value_collar = []
for s in S:
    if s > c:
        value_collar.append(c - S0)
    elif s < f:
        value_collar.append(f - S0)
    else:
        value_collar.append(s - S0)
value_collar = np.array(value_collar)

In [16]:
constraints = [y >= 0,
               np.tile(r, m).T@y == p[0],
               S.T@y == p[1],
               value_call_1.T@y == p[2],
               value_call_2.T@y == p[3],
               value_put_1.T@y == p[4],
               value_put_2.T@y == p[5],
               value_collar.T@y == p[6]]
obj_min = cp.Minimize(p_collar)
prob = cp.Problem(obj_min, constraints)
print("optimal lower bound is", prob.solve())

obj_max = cp.Maximize(p_collar)
prob = cp.Problem(obj_max, constraints)
print("optimal upper bound is", prob.solve())

optimal lower bound is 0.032619047619008155
optimal upper bound is 0.06495039682537881
